In [1]:
import pandas as pd
import pytz, datetime

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
lessons = pd.read_csv('/home/xenakrll/Documents/other/tetrix-test/tech_quality/lessons.csv', sep="|")
lessons.columns = ['lesson_id', 'event_id', 'subj', 'time']
lessons = lessons[:-1]
lessons.tail()

,lesson_id,event_id,subj,time
373,f579351b-177c-4c1c-850a-58bbda8b0b22,"48,887.00",hist,2020-01-15 13:45:00
374,056c3d91-dce7-4071-91d3-1de816ee63a3,"51,001.00",hist,2020-01-17 12:45:00
375,c64a1daf-d7f5-4c8c-ac9f-df634d77327b,"50,304.00",bio,2020-01-11 11:00:00
376,aa7702bc-5271-469c-a1c2-b7e814fd7e20,"49,880.00",bio,2020-01-16 15:00:00
377,8643fa00-217a-42ae-a27d-e6f47c0501f1,"55,449.00",bio,2020-01-19 12:15:00


In [4]:
lessons['time'] = pd.to_datetime(lessons['time'], format='%Y-%m-%d %H:%M:%S')
lessons['time'] = (
    lessons['time'].dt.tz_localize('UTC').      
    transform(lambda g: g.dt.tz_convert('Europe/Moscow'))) 
lessons['date'] = [d.date() for d in lessons['time']]
lessons['event_id'] = lessons['event_id'].astype(int)
lessons['event_id'] = lessons['event_id'].astype(str)
lessons['subj'] = lessons['subj'].str.strip()
lessons.head()

,lesson_id,event_id,subj,time,date
0,1e7bb408-cfef-4a9f-8328-351c9483a64c,38114,phys,2020-01-19 15:00:00+03:00,2020-01-19
1,6d8e59d9-a7c8-4bb3-8ff3-99cd07acdf1a,51568,it,2020-01-19 16:00:00+03:00,2020-01-19
2,62e1a078-33de-47c1-99d2-845b1daca56f,52790,hist,2020-01-19 16:00:00+03:00,2020-01-19
3,00fc6685-f53a-49bb-b960-5e0042fd3852,51341,phys,2020-01-17 15:00:00+03:00,2020-01-17
4,4cadf623-82e6-422f-a342-acf978302fb2,55048,phys,2020-01-19 17:00:00+03:00,2020-01-19


In [5]:
part = pd.read_csv('/home/xenakrll/Documents/other/tetrix-test/tech_quality/participants.csv', sep="|")
part.columns = ['event_id', 'uid']
part = part[:-1]
part['uid'] = part['uid'].str.strip()
part['event_id'] = part['event_id'].str.strip()
part.tail()

,event_id,uid
738,50304,ab7149fc-b650-4b6b-a368-0581f78276ea
739,49880,8476b612-223f-4c86-9b0b-14bc04759233
740,49880,582e6abb-dd9a-42fa-b441-455eac28327e
741,55449,ad7199dd-3bf6-4d6d-8ae7-448ecf90a4eb
742,55449,d3364cf3-46cc-4e64-ad11-70a069f7c049


In [6]:
users = pd.read_csv('/home/xenakrll/Documents/other/tetrix-test/tech_quality/users.csv', sep="|", skipinitialspace=True)
users.columns = ['uid', 'role']
users = users[:-1]
users['uid'] = users['uid'].str.strip() 
users.tail()

,uid,role
738,582e6abb-dd9a-42fa-b441-455eac28327e,pupil
739,ad7199dd-3bf6-4d6d-8ae7-448ecf90a4eb,tutor
740,d3364cf3-46cc-4e64-ad11-70a069f7c049,pupil
741,ad7199dd-3bf6-4d6d-8ae7-448ecf90a4eb,tutor
742,f76609c8-ba5d-4818-9b57-097debc90cab,pupil


In [7]:
quality = pd.read_csv('/home/xenakrll/Documents/other/tetrix-test/tech_quality/quality.csv', sep="|")
quality.columns = ['lesson_id', 'quality']
quality = quality[:-1]
quality.tail()

,lesson_id,quality
360,c64a1daf-d7f5-4c8c-ac9f-df634d77327b,5.00
361,aa7702bc-5271-469c-a1c2-b7e814fd7e20,5.00
362,8643fa00-217a-42ae-a27d-e6f47c0501f1,5.00
363,7a15af58-39d0-4207-bfdf-3fa176b7fb4e,5.00
364,7a15af58-39d0-4207-bfdf-3fa176b7fb4e,5.00


In [8]:
avg_grade = quality.groupby('lesson_id')['quality'].agg({'avg_grade': 'mean'}) #avg grade for each lesson
avg_grade.head()

/home/xenakrll/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


,avg_grade
lesson_id,
00fc6685-f53a-49bb-b960-5e0042fd3852,5.00
01f717d8-baaf-427d-a927-08c79713ae79,3.50
024d415a-4543-4e73-88c6-f4609e1bf1ed,4.50
03f4f4a8-c60c-41d7-a1cb-17b2d36ee183,5.00
056c3d91-dce7-4071-91d3-1de816ee63a3,4.00


In [9]:
lessons1 = lessons.merge(avg_grade, how='inner', left_on='lesson_id', right_index=True)
lessons1.head()

,lesson_id,event_id,subj,time,date,avg_grade
1,6d8e59d9-a7c8-4bb3-8ff3-99cd07acdf1a,51568,it,2020-01-19 16:00:00+03:00,2020-01-19,5.00
2,62e1a078-33de-47c1-99d2-845b1daca56f,52790,hist,2020-01-19 16:00:00+03:00,2020-01-19,5.00
3,00fc6685-f53a-49bb-b960-5e0042fd3852,51341,phys,2020-01-17 15:00:00+03:00,2020-01-17,5.00
4,4cadf623-82e6-422f-a342-acf978302fb2,55048,phys,2020-01-19 17:00:00+03:00,2020-01-19,5.00
6,e597a79a-3f68-4d15-bc77-c7000bcf8e52,53026,phys,2020-01-19 18:00:00+03:00,2020-01-19,5.00


In [10]:
participants = pd.merge(part,
                 users,
                 on='uid')
len(participants['uid'])

6619

In [11]:
participants = participants.drop_duplicates()
len(participants['uid'])

645

In [12]:
participants['event_id'] = participants['event_id'].astype(str)

In [13]:
lessons1 = lessons1.merge(participants, how='inner', on='event_id')

In [14]:
phys = lessons1[(lessons1['subj'] == 'phys')&(lessons1['role'] == 'tutor')]
phys = phys.reset_index(drop=True)
phys.head()

,lesson_id,event_id,subj,time,date,avg_grade,uid,role
0,00fc6685-f53a-49bb-b960-5e0042fd3852,51341,phys,2020-01-17 15:00:00+03:00,2020-01-17,5.00,30a19496-bdaf-461c-abbc-2709ae520201,tutor
1,4cadf623-82e6-422f-a342-acf978302fb2,55048,phys,2020-01-19 17:00:00+03:00,2020-01-19,5.00,43efce48-94b2-4412-857f-223d45969008,tutor
2,e597a79a-3f68-4d15-bc77-c7000bcf8e52,53026,phys,2020-01-19 18:00:00+03:00,2020-01-19,5.00,43efce48-94b2-4412-857f-223d45969008,tutor
3,ea6f0bf3-bcfc-4555-9b16-cc2cfe722e17,51645,phys,2020-01-19 17:00:00+03:00,2020-01-19,5.00,8fe03f08-8581-430c-a590-9888ab36deb3,tutor
4,2e1cef67-f1ae-4fe9-b1e1-1c5766d57d69,51645,phys,2020-01-12 17:00:00+03:00,2020-01-12,5.00,8fe03f08-8581-430c-a590-9888ab36deb3,tutor


In [15]:
phys_by_day = phys.groupby(['date','uid'],as_index=False).agg({'avg_grade': 'mean'})
phys_by_day

,date,uid,avg_grade
0,2020-01-11,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,5.00
1,2020-01-11,30a19496-bdaf-461c-abbc-2709ae520201,5.00
2,2020-01-11,43efce48-94b2-4412-857f-223d45969008,5.00
3,2020-01-11,603b8641-c6f6-4d89-ac89-88e50d45aa0d,4.83
4,2020-01-11,8fe03f08-8581-430c-a590-9888ab36deb3,4.50
5,2020-01-11,b37ccae8-fc31-4ad8-8f55-ca855b23fbf6,5.00
6,2020-01-11,c0c073c8-cb9c-4ebc-ac13-472c06029591,5.00
7,2020-01-12,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,5.00
8,2020-01-12,43efce48-94b2-4412-857f-223d45969008,5.00
9,2020-01-12,4df2832a-1d63-4453-9659-43993fc35996,5.00


In [16]:
phys_by_day = phys_by_day[phys_by_day['avg_grade'] == phys_by_day.groupby('date')['avg_grade'].transform('min')]

In [17]:
phys_by_day.reset_index(drop=True)

,date,uid,avg_grade
0,2020-01-11,8fe03f08-8581-430c-a590-9888ab36deb3,4.50
1,2020-01-12,696c838e-c054-4e9f-a51a-50bf5660f364,4.90
2,2020-01-13,2fa2ab62-f1b0-4036-872f-bcfd9a8686ff,5.00
3,2020-01-13,30a19496-bdaf-461c-abbc-2709ae520201,5.00
4,2020-01-13,696c838e-c054-4e9f-a51a-50bf5660f364,5.00
5,2020-01-13,b37ccae8-fc31-4ad8-8f55-ca855b23fbf6,5.00
6,2020-01-13,be676776-8366-4c71-8a35-d58014806eb5,5.00
7,2020-01-14,c6718d0e-976c-4d6c-b0e0-32c770776567,4.00
8,2020-01-15,603b8641-c6f6-4d89-ac89-88e50d45aa0d,5.00
9,2020-01-15,696c838e-c054-4e9f-a51a-50bf5660f364,5.00
